# Purpose
The goal of this notebook is to make a master CSV. 
# Unique ID
Unfortunately, many stations share the same name, which has forced me to ignore some in my analysis. This is meant to fix that. 

In [310]:
import pandas as pd
import re
import numpy as np

I'm going to look at the onboard by stations CSV since this seems to be a reliable source for stations (other CSVs like number of choice hires seems to leave out some stations).

In [311]:
staff = pd.read_csv("Staff_Size/Onboard_By_statyion_By_FY.csv")
staff = staff.drop(0).drop(1) # dropping first and second lines because these don't refer to actual stations.
staff.sample(10)

,ORGANIZATION,MAY-FY11,MAY-FY12,MAY-FY13,MAY-FY14,MAY-FY15,MAY-FY16
59,(V08) (675) MC ORLANDO FL,1991,2239.0,2464,2668,2914,3708
126,(V20) (531) MC BOISE ID,1022,1018.0,1037,1079,1164,1207
86,(V12) (585) MC IRON MOUNTAIN MI,604,613.0,634,655,667,657
100,(V16) (623) MC MUSKOGEE OK,1168,1150.0,1206,1257,1314,1318
65,(V09) (621) MC MOUNTAIN HOME TN,2087,2078.0,2120,2181,2280,2317
40,(V06) (652) MC RICHMOND VA,2296,2460.0,2631,2729,3001,3333
37,(V06) (565) MC FAYETTEVILLE NC,1142,1194.0,1283,1458,1596,1881
25,(V04) (562) MC ERIE PA,678,669.0,660,689,713,693
31,(V05) (512) HCS BALTIMORE MD,3062,3120.0,3194,3215,3269,3411
92,(V15) (657) HCS ST LOUIS MO,4436,4329.0,4441,4825,5076,5125


In [312]:
# taking this function from my analysis notebook to strip out the stations. Slight modifications
def get_station(new_string):
    try:
        query = r"^\((V\d\d)\) \((.*)\)"
        m = re.search(query, new_string)
        return m.group(2).upper()
    except:
        None
        
staff["Station"] = staff["ORGANIZATION"].apply(get_station)
#staff

In [313]:
staff["Station"].value_counts()



516    1
658    1
358    1
679    1
757    1
632    1
631    1
630    1
659    1
503    1
517    1
502    1
549    1
548    1
506    1
509    1
508    1
402    1
637    1
674    1
561    1
676    1
531    1
578    1
534    1
605    1
537    1
544    1
463    1
540    1
      ..
598    1
640    1
596    1
603    1
558    1
595    1
593    1
607    1
590    1
740    1
664    1
438    1
585    1
644    1
618    1
586    1
529    1
562    1
612    1
523    1
436    1
521    1
520    1
528    1
526    1
504    1
575    1
799    1
496    1
623    1
Name: Station, dtype: int64

Okay that's stange. There are no duplicates in this list. Could these all be unique IDs? If so I may not need to generate any new IDs and instead use these. 

### Bringing in Pending 2014 Wait Times

In [314]:
# A function to format 
def make_df(Type,Time,File):
    Title = "{}_{}".format(Type,Time)
    csv = pd.read_csv(File,
                               usecols=[0,1,5,24,25,26],
                               skiprows=[0], #skipping the original header
                               names=["Location","Appts_{}".format(Title),
                                      "%_Appts_Over_30_{}".format(Title),
                                      "PC_Wait_{}".format(Title),
                                      "SC_Wait_{}".format(Title),"MH_{}".format(Title)])
    csv["Station"] = csv["Location"].apply(get_station)
    return csv

In [315]:
Pending_1412 = make_df("Pending","1412","Wait_Time/Pending_Clean/14_12_Wait.csv")
Pending_1412.head()

,Location,Appts_Pending_1412,%_Appts_Over_30_Pending_1412,PC_Wait_Pending_1412,SC_Wait_Pending_1412,MH_Pending_1412,Station
0,"(V01) (402) Togus, ME","42,899",2.70%,2.51,4.07,2.77,402
1,"(V01) (405) White River Junction, VT","23,709",2.16%,3.92,3.95,0.79,405
2,"(V01) (518) Bedford, MA","8,105",5.17%,1.12,10.53,4.41,518
3,"(V01) (523) VA Boston HCS, MA","85,625",1.71%,1.66,3.53,3.92,523
4,"(V01) (608) Manchester, NH","19,511",2.64%,2.66,4.21,5.13,608


### Trying To Consolidate Staff And Pending

In [316]:
#Grabbing all stations in pending that have duplicates
multiple_pending_series = Pending_1412["Station"].value_counts()
multiple_pending_dataframe = multiple_pending_series.to_frame()
multiple_pending_list = multiple_pending_dataframe[multiple_pending_dataframe["Station"] > 1].index.tolist()

pending_unique = pd.DataFrame()
for item in multiple_pending_list:
    new = Pending_1412[Pending_1412["Station"]==item]
    pending_unique = pending_unique.append(new)
    
    
#grabbing all in staff that aren't in pending...  
in_staff_not_pending = [x for x in staff["Station"].tolist() if x not in Pending_1412["Station"].tolist()]
staff_unique = pd.DataFrame()
for item in in_staff_not_pending:
    new = staff[staff["Station"]==item]
    staff_unique = staff_unique.append(new)
    
# ...and including that in a dataframe with the uniques from 
for item in multiple_pending_list:
    new = staff[staff["Station"]==item]
    staff_unique = staff_unique.append(new)



#### Cleaning
Identified areas that need to be clean and stations that match. At this point I'm going to try and generate unique IDs and then I'm going to bring in other data to join. 

Many of the locations in station in the hiring csv don't match the wait time csv. 


In [317]:
# identified these as not stations, but facilities. Dropping from dataframe. 
to_drop = [18,29,50,59,66,77,87,96,105,114,123,144,152]
for index in to_drop:
    Pending_1412 = Pending_1412.drop(index)



### Generating Unique IDs

In [318]:
IDs = []
query = r"^\((V\d\d)\) \((\d\d\d)\) (.{2})"
#m = re.search(query, test)
#m.group(3)



for index, row in Pending_1412.iterrows():
    if row["Station"] not in IDs:
        print row["Location"]
        m = re.search(query, row["Location"])
        loc = m.group(3).lower()
        IDs.append(row["Station"])

    else:
        m = re.search(query, row["Location"])
        loc = m.group(3).lower()
        IDs.append(row["Station"]+loc)
Pending_1412["ID"] = IDs
Pending_1412

(V01) (402) Togus, ME
(V01) (405) White River Junction, VT
(V01) (518) Bedford, MA
(V01) (523) VA Boston HCS, MA
(V01) (608) Manchester, NH
(V01) (631) VA Central Western Massachusetts HCS
(V01) (650) Providence, RI
(V01) (689) VA Connecticut HCS, CT
(V02) (528) Albany, NY
(V03) (526) Bronx, NY
(V03) (561) New Jersey HCS, NJ
(V03) (620) VA Hudson Valley HCS, NY
(V03) (630) New York Harbor HCS, NY
(V03) (632) Northport, NY
(V04) (460) Wilmington, DE
(V04) (503) Altoona, PA
(V04) (529) Butler, PA
(V04) (540) Clarksburg, WV
(V04) (542) Coatesville, PA
(V04) (562) Erie, PA
(V04) (595) Lebanon, PA
(V04) (642) Philadelphia, PA
(V04) (646) Pittsburgh, PA
(V04) (693) Wilkes�ېBarre, PA
(V05) (512) Baltimore HCS, MD
(V05) (613) Martinsburg, WV
(V05) (688) Washington, DC
(V06) (517) Beckley, WV
(V06) (558) Durham, NC
(V06) (565) Fayetteville, NC
(V06) (590) Hampton, VA
(V06) (637) Asheville, NC
(V06) (652) Richmond, VA
(V06) (658) Salem, VA
(V06) (659) Salisbury, NC
(V07) (508) Atlanta, GA
(V07) 

,Location,Appts_Pending_1412,%_Appts_Over_30_Pending_1412,PC_Wait_Pending_1412,SC_Wait_Pending_1412,MH_Pending_1412,Station,ID
0,"(V01) (402) Togus, ME","42,899",2.70%,2.51,4.07,2.77,402,402
1,"(V01) (405) White River Junction, VT","23,709",2.16%,3.92,3.95,0.79,405,405
2,"(V01) (518) Bedford, MA","8,105",5.17%,1.12,10.53,4.41,518,518
3,"(V01) (523) VA Boston HCS, MA","85,625",1.71%,1.66,3.53,3.92,523,523
4,"(V01) (608) Manchester, NH","19,511",2.64%,2.66,4.21,5.13,608,608
5,(V01) (631) VA Central Western Massachusetts HCS,"22,602",9.26%,10.54,12.50,4.18,631,631
6,"(V01) (650) Providence, RI","38,893",4.12%,7.40,4.59,9.33,650,650
7,"(V01) (689) VA Connecticut HCS, CT","49,615",2.18%,2.35,4.15,3.92,689,689
8,"(V02) (528) Albany, NY","31,376",3.16%,2.33,3.64,4.29,528,528
9,"(V02) (528) Bath, NY","14,816",4.20%,3.35,7.70,4.97,528,528ba


In [319]:
#pd.read_csv("Wait_Time/Completed_Cleaned/Wait_Times_201409.csv")

In [320]:
#manually changing some IDs in staff based on shared location
staff["ID"] = staff["Station"]
staff["ID"][11] = "528"
staff["ID"][92] = "657st"
staff["ID"][12] = "528bu"
staff.head()

/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,ORGANIZATION,MAY-FY11,MAY-FY12,MAY-FY13,MAY-FY14,MAY-FY15,MAY-FY16,Station,ID
2,(V01) (402) HCS TOGUS ME,1293,1272.0,1248,1283,1335,1446,402,402
3,(V01) (405) MROC WHT RIVER JCT VT,852,853.0,881,921,1022,1177,405,405
4,(V01) (478) V1HCSBEDFORD MA,30,36.0,34,53,48,42,478,478
5,(V01) (518) MC BEDFORD MA,1313,1265.0,1245,1251,1297,1352,518,518
6,(V01) (523) HCS BOSTON MA,4022,4011.0,4041,4032,4123,4303,523,523


In [321]:
master = Pending_1412.merge(staff, on="ID", how="left")
master["Station"] = master["Station_x"]
master= master.drop("Station_x",1).drop("Station_y",1)
print master.columns

Index([u'Location', u'Appts_Pending_1412', u'%_Appts_Over_30_Pending_1412',
       u'PC_Wait_Pending_1412', u'SC_Wait_Pending_1412', u'MH_Pending_1412',
       u'ID', u'ORGANIZATION', u'MAY-FY11', u'MAY-FY12', u'MAY-FY13',
       u'MAY-FY14', u'MAY-FY15', u'MAY-FY16', u'Station'],
      dtype='object')


#### So what was the result of our merge?

In [322]:
def check_len(master):
    check = len(master) - 141
    if check == 0:
        print "Hurray! We didn't lose any rows."
    if check < 0:
        print "Uh oh. We lost {} rows!".format(str(abs(check)))
    if check > 0:
        print "we somehow...gained {} row? Well that's not right.".format(str(abs(check)))
check_len(master)

Hurray! We didn't lose any rows.


Great! Now I'm going to redefine ID so that it better matches with future dataframes.

In [323]:
ID = []
query = r"^\((V\d\d)\) \((\d\d\d)\) (.{2})"

for index, row in master.iterrows():
    m = re.search(query, row["Location"])
    loc = m.group(3).lower()
    ID.append(row["Station"]+loc)

master["ID"] = ID
master["ID"][72] = "537je" # edge case
master.head()

/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Location,Appts_Pending_1412,%_Appts_Over_30_Pending_1412,PC_Wait_Pending_1412,SC_Wait_Pending_1412,MH_Pending_1412,ID,ORGANIZATION,MAY-FY11,MAY-FY12,MAY-FY13,MAY-FY14,MAY-FY15,MAY-FY16,Station
0,"(V01) (402) Togus, ME","42,899",2.70%,2.51,4.07,2.77,402to,(V01) (402) HCS TOGUS ME,1293.0,1272.0,1248.0,1283.0,1335.0,1446.0,402
1,"(V01) (405) White River Junction, VT","23,709",2.16%,3.92,3.95,0.79,405wh,(V01) (405) MROC WHT RIVER JCT VT,852.0,853.0,881.0,921.0,1022.0,1177.0,405
2,"(V01) (518) Bedford, MA","8,105",5.17%,1.12,10.53,4.41,518be,(V01) (518) MC BEDFORD MA,1313.0,1265.0,1245.0,1251.0,1297.0,1352.0,518
3,"(V01) (523) VA Boston HCS, MA","85,625",1.71%,1.66,3.53,3.92,523va,(V01) (523) HCS BOSTON MA,4022.0,4011.0,4041.0,4032.0,4123.0,4303.0,523
4,"(V01) (608) Manchester, NH","19,511",2.64%,2.66,4.21,5.13,608ma,(V01) (608) MC MANCHESTER NH,663.0,683.0,691.0,686.0,759.0,803.0,608


## Bringing in Pending 2016

In [324]:
#functions to format 2016 
def get_station(new_string):
    try:
        query = r"^\((V\d\d)\) \(([0-9,A-Z]{3,5})"
        m = re.search(query, new_string)
        return m.group(2).upper()
    except:
        None
        
staff["Station"] = staff["ORGANIZATION"].apply(get_station)




def make_df_2016(Type,Time,File):
    Title = "{}_{}".format(Type,Time)
    csv = pd.read_csv(File,
                               usecols=[0,1,5,20,21,22],
                               skiprows=[0], #skipping the original header
                               names=["Location","Appts_{}".format(Title),
                                      "%_Appts_Over_30_{}".format(Title),
                                      "PC_Wait_{}".format(Title),
                                      "SC_Wait_{}".format(Title),"MH_{}".format(Title)])
    csv["Station"] = csv["Location"].apply(get_station)
    return csv
Pending_1610 = make_df_2016("Pending","1610","Wait_Time/Pending_Clean/16_10_Wait.csv")
#Pending_1610 = Pending_1610.drop(297).drop(369) # edge case causing issues, not a station anyway

In [325]:
def generate_IDs(df):
    ID = []
    for index, row in df.iterrows():
        #print index
        if len(row["Station"]) > 3:
            #print row["Station"]
            ID.append(None)
        else:
            try:
                #print index
                #print "entering else"
                #print "This is the location: "+row["Location"]
                query = r"^\((V\d\d)\) \((\d\d\d)\) (.{2})"
                m = re.search(query, row["Location"].upper())
                new_ID = row["Station"]+m.group(3).lower()
                if new_ID not in ID:
                    ID.append(new_ID)
                else:
                    ID.append(None)
                #print m.group(3).upper()
            except AttributeError:
                #print "AttributeError"
                ID.append(None)
    return ID
ID = generate_IDs(Pending_1610)
Pending_1610["ID"] = ID

In [326]:
Pending_1610 = Pending_1610.dropna(subset = ["ID"])
master = master.merge(Pending_1610,on="ID")
print master.columns
master["Station"] = master["Station_x"]
master["Location"] = master["Location_x"]
master= master.drop("Station_x",1).drop("Station_y",1).drop("Location_y",1).drop("Location_x",1)
master.head()

Index([u'Location_x', u'Appts_Pending_1412', u'%_Appts_Over_30_Pending_1412',
       u'PC_Wait_Pending_1412', u'SC_Wait_Pending_1412', u'MH_Pending_1412',
       u'ID', u'ORGANIZATION', u'MAY-FY11', u'MAY-FY12', u'MAY-FY13',
       u'MAY-FY14', u'MAY-FY15', u'MAY-FY16', u'Station_x', u'Location_y',
       u'Appts_Pending_1610', u'%_Appts_Over_30_Pending_1610',
       u'PC_Wait_Pending_1610', u'SC_Wait_Pending_1610', u'MH_Pending_1610',
       u'Station_y'],
      dtype='object')


,Appts_Pending_1412,%_Appts_Over_30_Pending_1412,PC_Wait_Pending_1412,SC_Wait_Pending_1412,MH_Pending_1412,ID,ORGANIZATION,MAY-FY11,MAY-FY12,MAY-FY13,MAY-FY14,MAY-FY15,MAY-FY16,Appts_Pending_1610,%_Appts_Over_30_Pending_1610,PC_Wait_Pending_1610,SC_Wait_Pending_1610,MH_Pending_1610,Station,Location
0,"42,899",2.70%,2.51,4.07,2.77,402to,(V01) (402) HCS TOGUS ME,1293.0,1272.0,1248.0,1283.0,1335.0,1446.0,"43,609",4.18%,7.82,5.67,3.62,402,"(V01) (402) Togus, ME"
1,"23,709",2.16%,3.92,3.95,0.79,405wh,(V01) (405) MROC WHT RIVER JCT VT,852.0,853.0,881.0,921.0,1022.0,1177.0,"21,091",3.83%,6.09,7.09,1.86,405,"(V01) (405) White River Junction, VT"
2,"8,105",5.17%,1.12,10.53,4.41,518be,(V01) (518) MC BEDFORD MA,1313.0,1265.0,1245.0,1251.0,1297.0,1352.0,"14,456",7.54%,0.60,11.21,3.07,518,"(V01) (518) Bedford, MA"
3,"85,625",1.71%,1.66,3.53,3.92,523va,(V01) (523) HCS BOSTON MA,4022.0,4011.0,4041.0,4032.0,4123.0,4303.0,"84,373",4.43%,14.46,6.76,3.08,523,"(V01) (523) VA Boston HCS, MA"
4,"19,511",2.64%,2.66,4.21,5.13,608ma,(V01) (608) MC MANCHESTER NH,663.0,683.0,691.0,686.0,759.0,803.0,"25,424",4.09%,5.75,4.58,5.21,608,"(V01) (608) Manchester, NH"


In [327]:
check_len(master)

Hurray! We didn't lose any rows.


## Bringing in Complete 2016

In [328]:
def get_station(new_string):
    try:
        query = r"^\((V\d\d)\) \(([0-9,A-Z]{3,5})"
        m = re.search(query, new_string)
        return m.group(2).upper()
    except:
        None
        
staff["Station"] = staff["ORGANIZATION"].apply(get_station)




def make_df_2016(Type,Time,File):
    Title = "{}_{}".format(Type,Time)
    csv = pd.read_csv(File,
                               usecols=[0,1,5,13,14,15],
                               skiprows=[0], #skipping the original header
                               names=["Location","Appts_{}".format(Title),
                                      "%_Appts_Over_30_{}".format(Title),
                                      "PC_Wait_{}".format(Title),
                                      "SC_Wait_{}".format(Title),"MH_{}".format(Title)])
    csv["Station"] = csv["Location"].apply(get_station)
    return csv
Complete_1608 = make_df_2016("Complete","1608","Wait_Time/Completed_Cleaned/16_08_Wait.csv")
Complete_1608.head()

,Location,Appts_Complete_1608,%_Appts_Over_30_Complete_1608,PC_Wait_Complete_1608,SC_Wait_Complete_1608,MH_Complete_1608,Station
0,"(V01) (402) Togus, ME","28,683",1.34%,4.96,2.73,1.87,402
1,(V01) (402) Togus VAMC,"17,464",1.63%,5.96,2.66,2.59,402
2,(V01) (402GA) Aroostook County\r(Caribou),756,4.23%,8.84,0.00,3.41,402GA
3,(V01) (402GB) Calais,379,0.00%,1.43,0.00,0.36,402GB
4,(V01) (402GC) Rumford,428,0.00%,1.31,0.00,1.11,402GC


In [329]:
ID = generate_IDs(Complete_1608)
Complete_1608["ID"] = ID
Complete_1608 = Complete_1608.dropna(subset = ["ID"])
master = master.merge(Complete_1608,on="ID")
master["Station"] = master["Station_x"]
master["Location"] = master["Location_x"]
master= master.drop("Station_x",1).drop("Station_y",1).drop("Location_y",1).drop("Location_x",1)

In [330]:
master.columns

Index([u'Appts_Pending_1412', u'%_Appts_Over_30_Pending_1412',
       u'PC_Wait_Pending_1412', u'SC_Wait_Pending_1412', u'MH_Pending_1412',
       u'ID', u'ORGANIZATION', u'MAY-FY11', u'MAY-FY12', u'MAY-FY13',
       u'MAY-FY14', u'MAY-FY15', u'MAY-FY16', u'Appts_Pending_1610',
       u'%_Appts_Over_30_Pending_1610', u'PC_Wait_Pending_1610',
       u'SC_Wait_Pending_1610', u'MH_Pending_1610', u'Appts_Complete_1608',
       u'%_Appts_Over_30_Complete_1608', u'PC_Wait_Complete_1608',
       u'SC_Wait_Complete_1608', u'MH_Complete_1608', u'Station', u'Location'],
      dtype='object')

In [331]:
check_len(master)

Hurray! We didn't lose any rows.


In [332]:
# Reference for duplicates I need to check out. 
master[master.duplicated(subset="Station", keep=False)][["Location","Station","ID"]]

,Location,Station,ID
8,"(V02) (528) Albany, NY",528,528al
9,"(V02) (528) Bath, NY",528,528ba
10,"(V02) (528) Western New York, NY",528,528we
11,"(V02) (528) Canandaigua, NY",528,528ca
12,"(V02) (528) Syracuse, NY",528,528sy
79,"(V15) (589) Columbia, MO",589,589co
80,"(V15) (589) Kansas City, MO",589,589ka
81,"(V15) (589) Eastern KS HCS, KS",589,589ea
82,"(V15) (589) Wichita, KS",589,589wi
83,"(V15) (657) Marion, IL",657,657ma


## Bring in Hiring

I'm going to have an issue dealing with the stations with the same ID. 

I made a new CSV called Hiring_Clean where I put the appropriate ID in a new ID column. I put ignore for those that were impossible to join accurately. 

In [333]:
master.columns

Index([u'Appts_Pending_1412', u'%_Appts_Over_30_Pending_1412',
       u'PC_Wait_Pending_1412', u'SC_Wait_Pending_1412', u'MH_Pending_1412',
       u'ID', u'ORGANIZATION', u'MAY-FY11', u'MAY-FY12', u'MAY-FY13',
       u'MAY-FY14', u'MAY-FY15', u'MAY-FY16', u'Appts_Pending_1610',
       u'%_Appts_Over_30_Pending_1610', u'PC_Wait_Pending_1610',
       u'SC_Wait_Pending_1610', u'MH_Pending_1610', u'Appts_Complete_1608',
       u'%_Appts_Over_30_Complete_1608', u'PC_Wait_Complete_1608',
       u'SC_Wait_Complete_1608', u'MH_Complete_1608', u'Station', u'Location'],
      dtype='object')

In [334]:
Hiring = pd.read_csv("Hiring/Hiring_Clean.csv")
Hiring = Hiring[Hiring["ID"]!= "Ignore"] 
Hiring_summed = pd.pivot_table(Hiring,index=["Station"],values=["NbrEmps"],aggfunc=np.sum).reset_index()
Hiring_summed = Hiring_summed.rename(columns = {"NbrEmps":"Choice_Hires"})

In [335]:
master.columns

Index([u'Appts_Pending_1412', u'%_Appts_Over_30_Pending_1412',
       u'PC_Wait_Pending_1412', u'SC_Wait_Pending_1412', u'MH_Pending_1412',
       u'ID', u'ORGANIZATION', u'MAY-FY11', u'MAY-FY12', u'MAY-FY13',
       u'MAY-FY14', u'MAY-FY15', u'MAY-FY16', u'Appts_Pending_1610',
       u'%_Appts_Over_30_Pending_1610', u'PC_Wait_Pending_1610',
       u'SC_Wait_Pending_1610', u'MH_Pending_1610', u'Appts_Complete_1608',
       u'%_Appts_Over_30_Complete_1608', u'PC_Wait_Complete_1608',
       u'SC_Wait_Complete_1608', u'MH_Complete_1608', u'Station', u'Location'],
      dtype='object')

In [336]:
master = master.merge(Hiring_summed, on="Station", how="left")
master.head()

,Appts_Pending_1412,%_Appts_Over_30_Pending_1412,PC_Wait_Pending_1412,SC_Wait_Pending_1412,MH_Pending_1412,ID,ORGANIZATION,MAY-FY11,MAY-FY12,MAY-FY13,...,SC_Wait_Pending_1610,MH_Pending_1610,Appts_Complete_1608,%_Appts_Over_30_Complete_1608,PC_Wait_Complete_1608,SC_Wait_Complete_1608,MH_Complete_1608,Station,Location,Choice_Hires
0,"42,899",2.70%,2.51,4.07,2.77,402to,(V01) (402) HCS TOGUS ME,1293.0,1272.0,1248.0,...,5.67,3.62,"28,683",1.34%,4.96,2.73,1.87,402,"(V01) (402) Togus, ME",23.0
1,"23,709",2.16%,3.92,3.95,0.79,405wh,(V01) (405) MROC WHT RIVER JCT VT,852.0,853.0,881.0,...,7.09,1.86,"21,581",1.77%,3.62,4.08,1.32,405,"(V01) (405) White River Junction, VT",24.0
2,"8,105",5.17%,1.12,10.53,4.41,518be,(V01) (518) MC BEDFORD MA,1313.0,1265.0,1245.0,...,11.21,3.07,"13,170",1.15%,0.43,6.01,0.76,518,"(V01) (518) Bedford, MA",5.0
3,"85,625",1.71%,1.66,3.53,3.92,523va,(V01) (523) HCS BOSTON MA,4022.0,4011.0,4041.0,...,6.76,3.08,"46,306",2.71%,7.47,5.75,2.17,523,"(V01) (523) VA Boston HCS, MA",28.0
4,"19,511",2.64%,2.66,4.21,5.13,608ma,(V01) (608) MC MANCHESTER NH,663.0,683.0,691.0,...,4.58,5.21,"18,849",1.60%,4.99,2.93,2.94,608,"(V01) (608) Manchester, NH",19.0


In [337]:
check_len(master)

Hurray! We didn't lose any rows.


## Bring in Leadership

Also creating a version of the CSV to deal with duplicate stations.

In [338]:
Leadership = pd.read_csv("Leadership/Leadership_cleaned.csv")
Leadership = Leadership[Leadership["ID"] != "Ignore"]
Leadership["Vacancy"] = Leadership["Acting/Detailed"] == "VACANT"
Leadership["Interim"] = Leadership["Acting/Detailed"] != "VACANT"


# For easy merging


for_dups = []
for index, row in Leadership.iterrows():
    if type(row["ID"]) == str:
        for_dups.append(row["ID"])
    else:
        for_dups.append(row["Station"])
Leadership["for_dups"] = for_dups

Leadership_summed = pd.pivot_table(Leadership, index=["for_dups"],values=["Vacancy","Interim"],aggfunc=np.sum).reset_index()
#Leadership_summed = Leadership_summed.merge(Leadership[["Station","ID"]], on="Station")





dup_list = master[master.duplicated(subset="Station",keep=False)]["ID"].tolist()
dup_list.append("612n.")


for_dups = []
for index, row in master.iterrows():
    if row["ID"] in dup_list:
        for_dups.append(row["ID"])
    else:
        for_dups.append(row["Station"])
master["for_dups"] = for_dups






In [339]:
master = master.merge(Leadership_summed, on="for_dups", how="left")
master["Interim"] = master["Interim"].fillna(0)
master["Vacancy"] = master["Vacancy"].fillna(0)

In [340]:
check_len(master)

Hurray! We didn't lose any rows.


In [341]:
master.head()

,Appts_Pending_1412,%_Appts_Over_30_Pending_1412,PC_Wait_Pending_1412,SC_Wait_Pending_1412,MH_Pending_1412,ID,ORGANIZATION,MAY-FY11,MAY-FY12,MAY-FY13,...,%_Appts_Over_30_Complete_1608,PC_Wait_Complete_1608,SC_Wait_Complete_1608,MH_Complete_1608,Station,Location,Choice_Hires,for_dups,Interim,Vacancy
0,"42,899",2.70%,2.51,4.07,2.77,402to,(V01) (402) HCS TOGUS ME,1293.0,1272.0,1248.0,...,1.34%,4.96,2.73,1.87,402,"(V01) (402) Togus, ME",23.0,402,0.0,1.0
1,"23,709",2.16%,3.92,3.95,0.79,405wh,(V01) (405) MROC WHT RIVER JCT VT,852.0,853.0,881.0,...,1.77%,3.62,4.08,1.32,405,"(V01) (405) White River Junction, VT",24.0,405,0.0,0.0
2,"8,105",5.17%,1.12,10.53,4.41,518be,(V01) (518) MC BEDFORD MA,1313.0,1265.0,1245.0,...,1.15%,0.43,6.01,0.76,518,"(V01) (518) Bedford, MA",5.0,518,0.0,1.0
3,"85,625",1.71%,1.66,3.53,3.92,523va,(V01) (523) HCS BOSTON MA,4022.0,4011.0,4041.0,...,2.71%,7.47,5.75,2.17,523,"(V01) (523) VA Boston HCS, MA",28.0,523,0.0,0.0
4,"19,511",2.64%,2.66,4.21,5.13,608ma,(V01) (608) MC MANCHESTER NH,663.0,683.0,691.0,...,1.60%,4.99,2.93,2.94,608,"(V01) (608) Manchester, NH",19.0,608,0.0,0.0


In [342]:
master.columns

Index([u'Appts_Pending_1412', u'%_Appts_Over_30_Pending_1412',
       u'PC_Wait_Pending_1412', u'SC_Wait_Pending_1412', u'MH_Pending_1412',
       u'ID', u'ORGANIZATION', u'MAY-FY11', u'MAY-FY12', u'MAY-FY13',
       u'MAY-FY14', u'MAY-FY15', u'MAY-FY16', u'Appts_Pending_1610',
       u'%_Appts_Over_30_Pending_1610', u'PC_Wait_Pending_1610',
       u'SC_Wait_Pending_1610', u'MH_Pending_1610', u'Appts_Complete_1608',
       u'%_Appts_Over_30_Complete_1608', u'PC_Wait_Complete_1608',
       u'SC_Wait_Complete_1608', u'MH_Complete_1608', u'Station', u'Location',
       u'Choice_Hires', u'for_dups', u'Interim', u'Vacancy'],
      dtype='object')

# Export to CSV

In [344]:
master.to_csv("Master/Master.csv")